In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

sc = SparkSession.builder.appName("TitanicTrainExercise")

##Step1
train_DF =  spark.read.csv('/FileStore/tables/train.csv', header="true", inferSchema="true")

##Step2:
#1 Print the dataset and verify that the schema contains all the variables.
train_DF.printSchema()



root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [2]:
#2 Print the first 10 rows from the dataset.
train_DF.show(10,False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass|Name |Sex |Age |SibSp|Parch|Ticket |Fare |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
1 |0 |3 |Braund, Mr. Owen Harris |male |22.0|1 |0 |A/5 21171 |7.25 |null |S |
2 |1 |1 |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1 |0 |PC 17599 |71.2833|C85 |C |
3 |1 |3 |Heikkinen, Miss. Laina |female|26.0|0 |0 |STON/O2. 3101282|7.925 |null |S |
4 |1 |1 |Futrelle, Mrs. Jacques Heath (Lily May Peel) |female|35.0|1 |0 |113803 |53.1 |C123 |S |
5 |0 |3 |Allen, Mr. William Henry |male |35.0|0 |0 |373450 |8.05 |null |S |
6 |0 |3 |Moran, Mr. James |male |null|0 |0 |330877 |8.4583 |null |Q |
7 |0 |1 |McCarthy, Mr. Timothy J |male |54.0|0 |0 |17463 |51.8625|E46 |S |
8 |0 |3 |Palsson, Master. Gosta Leonard |male |2.0 |3 |1 |349909 |21.075 |null |S |
9 |1 |3 |Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg) |female|27.0|0 |2 |347742 |11.1333|null |S |
10 |1 |2 |Nasser, Mrs. Nicholas (Adele Achem) |female|14.0|1 |0 |237736 |30.0708|null |C |
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 10 rows

In [3]:
#3 Obtain summary statistics for all variables in the dataframe. Pay attention to whether there are missing data as well as whether the field appears to be continuous or discrete.

train_DF.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
summary| PassengerId| Survived| Pclass| Name| Sex| Age| SibSp| Parch| Ticket| Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
 count| 891| 891| 891| 891| 891| 714| 891| 891| 891| 891| 204| 889|
 mean| 446.0| 0.3838383838383838| 2.308641975308642| null| null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.2042079685746| null| null|
 stddev|257.3538420152301|0.48659245426485753|0.8360712409770491| null| null|14.526497332334035|1.1027434322934315| 0.8060572211299488|471609.26868834975|49.69342859718089| null| null|
 min| 1| 0| 1|"Andersson, Mr. A...|female| 0.42| 0| 0| 110152| 0.0| A10| C|
 25%| 223| 0| 2| null| null| 20.0| 0| 0| 19996.0| 7.8958| null| null|
 50%| 446| 0| 3| null| null| 28.0| 0| 0| 236171.0| 14.4542| null| null|
 75%| 669| 1| 3| null| null| 38.0| 1| 0| 347743.0| 31.0| null| null|
 max| 891| 1| 3|van Melkebeke, Mr...| male| 80.0| 8| 6| WE/P 5735| 512.3292| T| S|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+

In [4]:
#4 For each of the string columns (except name and ticket), print the count of the 10 most frequent values ordered by descending order of frequency. 

train_DF.createOrReplaceTempView('traintable')
sex_f = spark.sql("select sex , count(*)  from traintable group by sex order by count(*) desc limit 10")
sex_f.show()

ticket_f = spark.sql("select ticket,count(*) from traintable group by ticket order by count(*) desc limit 10")
ticket_f.show()

cabin_f = spark.sql("select cabin,count(*) from traintable group by cabin order by count(*) desc limit 10")
cabin_f.show()

+------+--------+
 sex|count(1)|
+------+--------+
 male| 577|
female| 314|
+------+--------+

+------------+--------+
 ticket|count(1)|
+------------+--------+
 1601| 7|
 347082| 7|
 CA. 2343| 7|
 347088| 6|
 CA 2144| 6|
 3101295| 6|
S.O.C. 14879| 5|
 382652| 5|
 347077| 4|
 349909| 4|
+------------+--------+

+-----------+--------+
 cabin|count(1)|
+-----------+--------+
 null| 687|
 B96 B98| 4|
 G6| 4|
C23 C25 C27| 4|
 F2| 3|
 C22 C26| 3|
 D| 3|
 E101| 3|
 F33| 3|
 C65| 2|
+-----------+--------+

In [5]:
#Feature columns : Sex,Embarked, Pclass, SibSp, Parch, Age, Fare
##Step 3

#Select all feature columns you plan to use in addition to the target variable (i.e., ‘Survived’) and covert all numerical columns into double data type.

doubleCol = ['Pclass','SibSp','Parch']

from pyspark.sql.types import DoubleType

train_DF = train_DF.withColumn('Pclass',train_DF.Pclass.cast(DoubleType())).withColumn('SibSp',train_DF.SibSp.cast(DoubleType())).withColumn('Parch',train_DF.Parch.cast(DoubleType())).select('Sex','Embarked','Pclass','SibSp','Parch','Age','Fare','Survived')


In [6]:
#2 Replace the missing values in the Age column with the mean value. Create also a new variable (e.g., ‘AgeNA’) indicating whether the value of age was missing or not.
mean_age = train_DF.agg({"Age": "avg"})
mean_age.show()

train_DF= train_DF.fillna({'Age':29.7})
train_DF.show(30,False)


+-----------------+
 avg(Age)|
+-----------------+
29.69911764705882|
+-----------------+

+------+--------+------+-----+-----+----+-------+--------+
Sex |Embarked|Pclass|SibSp|Parch|Age |Fare |Survived|
+------+--------+------+-----+-----+----+-------+--------+
male |S |3.0 |1.0 |0.0 |22.0|7.25 |0 |
female|C |1.0 |1.0 |0.0 |38.0|71.2833|1 |
female|S |3.0 |0.0 |0.0 |26.0|7.925 |1 |
female|S |1.0 |1.0 |0.0 |35.0|53.1 |1 |
male |S |3.0 |0.0 |0.0 |35.0|8.05 |0 |
male |Q |3.0 |0.0 |0.0 |29.7|8.4583 |0 |
male |S |1.0 |0.0 |0.0 |54.0|51.8625|0 |
male |S |3.0 |3.0 |1.0 |2.0 |21.075 |0 |
female|S |3.0 |0.0 |2.0 |27.0|11.1333|1 |
female|C |2.0 |1.0 |0.0 |14.0|30.0708|1 |
female|S |3.0 |1.0 |1.0 |4.0 |16.7 |1 |
female|S |1.0 |0.0 |0.0 |58.0|26.55 |1 |
male |S |3.0 |0.0 |0.0 |20.0|8.05 |0 |
male |S |3.0 |1.0 |5.0 |39.0|31.275 |0 |
female|S |3.0 |0.0 |0.0 |14.0|7.8542 |0 |
female|S |2.0 |0.0 |0.0 |55.0|16.0 |1 |
male |Q |3.0 |4.0 |1.0 |2.0 |29.125 |0 |
male |S |2.0 |0.0 |0.0 |29.7|13.0 |1 |
female|S |3.0 |1.0 |0.0 |31.0|18.0 |0 |
female|C |3.0 |0.0 |0.0 |29.7|7.225 |1 |
male |S |2.0 |0.0 |0.0 |35.0|26.0 |0 |
male |S |2.0 |0.0 |0.0 |34.0|13.0 |1 |
female|Q |3.0 |0.0 |0.0 |15.0|8.0292 |1 |
male |S |1.0 |0.0 |0.0 |28.0|35.5 |1 |
female|S |3.0 |3.0 |1.0 |8.0 |21.075 |0 |
female|S |3.0 |1.0 |5.0 |38.0|31.3875|1 |
male |C |3.0 |0.0 |0.0 |29.7|7.225 |0 |
male |S |1.0 |3.0 |2.0 |19.0|263.0 |0 |
female|Q |3.0 |0.0 |0.0 |29.7|7.8792 |1 |
male |S |3.0 |0.0 |0.0 |29.7|7.8958 |0 |
+------+--------+------+-----+-----+----+-------+--------+
only showing top 30 rows

In [7]:
def valueToCategory(value):
    if value == 29.7: 
        return True
    else: 
        return False

udfValueToCategory = udf(valueToCategory, StringType())
train_DF_C = train_DF.withColumn("AgeNA", udfValueToCategory("Age"))
train_DF_C.show(30)

+------+--------+------+-----+-----+----+-------+--------+-----+
 Sex|Embarked|Pclass|SibSp|Parch| Age| Fare|Survived|AgeNA|
+------+--------+------+-----+-----+----+-------+--------+-----+
 male| S| 3.0| 1.0| 0.0|22.0| 7.25| 0|false|
female| C| 1.0| 1.0| 0.0|38.0|71.2833| 1|false|
female| S| 3.0| 0.0| 0.0|26.0| 7.925| 1|false|
female| S| 1.0| 1.0| 0.0|35.0| 53.1| 1|false|
 male| S| 3.0| 0.0| 0.0|35.0| 8.05| 0|false|
 male| Q| 3.0| 0.0| 0.0|29.7| 8.4583| 0| true|
 male| S| 1.0| 0.0| 0.0|54.0|51.8625| 0|false|
 male| S| 3.0| 3.0| 1.0| 2.0| 21.075| 0|false|
female| S| 3.0| 0.0| 2.0|27.0|11.1333| 1|false|
female| C| 2.0| 1.0| 0.0|14.0|30.0708| 1|false|
female| S| 3.0| 1.0| 1.0| 4.0| 16.7| 1|false|
female| S| 1.0| 0.0| 0.0|58.0| 26.55| 1|false|
 male| S| 3.0| 0.0| 0.0|20.0| 8.05| 0|false|
 male| S| 3.0| 1.0| 5.0|39.0| 31.275| 0|false|
female| S| 3.0| 0.0| 0.0|14.0| 7.8542| 0|false|
female| S| 2.0| 0.0| 0.0|55.0| 16.0| 1|false|
 male| Q| 3.0| 4.0| 1.0| 2.0| 29.125| 0|false|
 male| S| 2.0| 0.0| 0.0|29.7| 13.0| 1| true|
female| S| 3.0| 1.0| 0.0|31.0| 18.0| 0|false|
female| C| 3.0| 0.0| 0.0|29.7| 7.225| 1| true|
 male| S| 2.0| 0.0| 0.0|35.0| 26.0| 0|false|
 male| S| 2.0| 0.0| 0.0|34.0| 13.0| 1|false|
female| Q| 3.0| 0.0| 0.0|15.0| 8.0292| 1|false|
 male| S| 1.0| 0.0| 0.0|28.0| 35.5| 1|false|
female| S| 3.0| 3.0| 1.0| 8.0| 21.075| 0|false|
female| S| 3.0| 1.0| 5.0|38.0|31.3875| 1|false|
 male| C| 3.0| 0.0| 0.0|29.7| 7.225| 0| true|
 male| S| 1.0| 3.0| 2.0|19.0| 263.0| 0|false|
female| Q| 3.0| 0.0| 0.0|29.7| 7.8792| 1| true|
 male| S| 3.0| 0.0| 0.0|29.7| 7.8958| 0| true|
+------+--------+------+-----+-----+----+-------+--------+-----+
only showing top 30 rows

In [8]:
# Print the revised dataframe and recalculate the summary statistics.
train_DF_C.summary().show()


+-------+------+--------+------------------+------------------+-------------------+------------------+-----------------+-------------------+-----+
summary| Sex|Embarked| Pclass| SibSp| Parch| Age| Fare| Survived|AgeNA|
+-------+------+--------+------------------+------------------+-------------------+------------------+-----------------+-------------------+-----+
 count| 891| 889| 891| 891| 891| 891| 891| 891| 891|
 mean| null| null| 2.308641975308642|0.5230078563411896|0.38159371492704824| 29.69929292929302| 32.2042079685746| 0.3838383838383838| null|
 stddev| null| null|0.8360712409770491|1.1027434322934315| 0.8060572211299488|13.002015230774303|49.69342859718089|0.48659245426485753| null|
 min|female| C| 1.0| 0.0| 0.0| 0.42| 0.0| 0|false|
 25%| null| null| 2.0| 0.0| 0.0| 22.0| 7.8958| 0| null|
 50%| null| null| 3.0| 0.0| 0.0| 29.7| 14.4542| 0| null|
 75%| null| null| 3.0| 1.0| 0.0| 35.0| 31.0| 1| null|
 max| male| S| 3.0| 8.0| 6.0| 80.0| 512.3292| 1| true|
+-------+------+--------+------------------+------------------+-------------------+------------------+-----------------+-------------------+-----+